In [2]:
!pip install h5py


In [3]:
import h5py


In [5]:
import numpy as np


In [7]:
import h5py
from google.colab import drive
drive.mount('/content/drive')

# Load the file and print the available keys
with h5py.File('/content/drive/MyDrive/Train.h5', 'r') as train_dataset:
    print("Keys in Train.h5:", list(train_dataset.keys()))

with h5py.File('/content/drive/MyDrive/Test.h5', 'r') as test_dataset:
    print("Keys in Test.h5:", list(test_dataset.keys()))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Keys in Train.h5: ['images', 'labels']
Keys in Test.h5: ['images', 'labels']


In [8]:
import h5py
import numpy as np
from google.colab import drive

# Mount Google Drive (if not already mounted)
drive.mount('/content/drive', force_remount=True)

# Load the training and test data
with h5py.File('/content/drive/MyDrive/Train.h5', 'r') as train_dataset:
    train_images = np.array(train_dataset['images'][:])
    train_labels = np.array(train_dataset['labels'][:])

with h5py.File('/content/drive/MyDrive/Test.h5', 'r') as test_dataset:
    test_images = np.array(test_dataset['images'][:])
    test_labels = np.array(test_dataset['labels'][:])

print("Training and test data loaded successfully.")


Mounted at /content/drive
Training and test data loaded successfully.


In [9]:
# Flatten and normalize the images
train_images = train_images.reshape(train_images.shape[0], -1) / 255.0
test_images = test_images.reshape(test_images.shape[0], -1) / 255.0

# Convert labels to one-hot encoding
num_classes = len(np.unique(train_labels))
train_labels_one_hot = np.eye(num_classes)[train_labels]
test_labels_one_hot = np.eye(num_classes)[test_labels]

print("Data preprocessing completed.")


Data preprocessing completed.


In [10]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # for numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)


In [11]:
def compute_cost(y_true, y_pred):
    m = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / m  # adding epsilon for numerical stability


4. Gradient Descent



In [12]:
def gradient_descent(X, y, weights, bias, learning_rate):
    m = X.shape[0]

    # Forward propagation
    z = np.dot(X, weights) + bias
    y_pred = softmax(z)

    # Compute gradients
    dz = y_pred - y
    dw = np.dot(X.T, dz) / m
    db = np.sum(dz) / m

    # Update parameters
    weights -= learning_rate * dw
    bias -= learning_rate * db

    return weights, bias


## 5. Model **Training**





In [13]:
# Initialize parameters
num_features = train_images.shape[1]
weights = np.random.randn(num_features, num_classes) * 0.01
bias = np.zeros((1, num_classes))

# Training parameters
learning_rate = 0.1
num_epochs = 1000

# Training loop
for epoch in range(num_epochs):
    # Update weights and bias using gradient descent
    weights, bias = gradient_descent(train_images, train_labels_one_hot, weights, bias, learning_rate)

    # Calculate cost every 100 epochs
    if epoch % 100 == 0:
        z = np.dot(train_images, weights) + bias
        y_pred = softmax(z)
        cost = compute_cost(train_labels_one_hot, y_pred)
        print(f"Epoch {epoch}, Cost: {cost}")


Epoch 0, Cost: 14.201252185712509
Epoch 100, Cost: 10.354243833310864
Epoch 200, Cost: 10.791001213259358
Epoch 300, Cost: 11.786996801846858
Epoch 400, Cost: 10.659673020736529
Epoch 500, Cost: 12.00868028569123
Epoch 600, Cost: 8.0302073109131
Epoch 700, Cost: 9.270541009571254
Epoch 800, Cost: 8.251757450166458
Epoch 900, Cost: 9.713520223699147


In [14]:
# Function to make predictions
def predict(X, weights, bias):
    z = np.dot(X, weights) + bias
    y_pred = softmax(z)
    return np.argmax(y_pred, axis=1)

# Predictions on test set
test_predictions = predict(test_images, weights, bias)

# Calculate accuracy
accuracy = np.mean(test_predictions == test_labels) * 100
print(f"Test Set Accuracy: {accuracy:.2f}%")


Test Set Accuracy: 36.67%
